In [1]:
from svnds import PATH_DATA

import pickle
import os
import numpy as np

import eazy

from astropy.table import Table

<div class="alert alert-block alert-warning"> `eazy-py` should be installed before running this notebook </div>

* Install [eazy-py](https://github.com/gbrammer/eazy-py)

## .RES for filters in EAZY

Load 7DS filters & transform them into a .res file.

In [2]:
lambda_7ds = np.arange(4000., 9000., 125)

path_filter = os.path.join(PATH_DATA, "filters/SDS/filters_corrected")

with open(path_filter, 'rb') as fr:
    filters_corrected = pickle.load(fr)


In [14]:
# from svnds.info_PS import NAMES, FILTERS, COL_WAVE, COL_RESP
from svnds.info_VIKING import NAMES, FILTERS, COL_WAVE, COL_RESP

In [15]:
NAMES

['Z_V', 'Y_V', 'J_V', 'H_V', 'Ks_V']

In [16]:
fsets = ''

# 40 7DS filters
for ii, wl_cen in enumerate(lambda_7ds):

    wave_cen = f'{int(wl_cen):d}'
    
    wave = filters_corrected['wave_' + wave_cen] * 1e4
    resp = filters_corrected['resp_' + wave_cen]
        
    fset = eazy.filters.FilterDefinition(wave = wave, throughput = resp, name = f'F{int(wl_cen):d}W250')
    fsets = fsets + fset.for_filter_file() + '\n'

# # PS filters
# for ii, wl_cen in enumerate(NAMES):

#     wave_cen = wl_cen

#     wave = FILTERS[ii][COL_WAVE] * 10 #nm to AA
#     resp = FILTERS[ii][COL_RESP]
        
#     fset = eazy.filters.FilterDefinition(wave = wave, throughput = resp, name = 'F_' + wl_cen)
#     fsets = fsets + fset.for_filter_file() + '\n'

# VIKING filters
for ii, wl_cen in enumerate(NAMES):

    wave_cen = wl_cen

    wave = FILTERS[ii][COL_WAVE] * 10 #nm to AA
    resp = FILTERS[ii][COL_RESP]
        
    fset = eazy.filters.FilterDefinition(wave = wave, throughput = resp, name = 'F_' + wl_cen)
    fsets = fsets + fset.for_filter_file() + '\n'

with open(os.path.join(PATH_DATA, "/home/ehko/photz_for_7ds/svnds/data/filters/VIKING/Masters_VIKING.FILTER.RES"), 'w') as f:
    f.write(fsets)

## .cat for an input catalog in EAZY

In [2]:
lambda_7ds = np.arange(4000., 9000., 125)

from svnds.info_SPHEREx import SPHEREx_lambda_min, SPHEREx_lambda_max, SPHEREx_R

lmin = SPHEREx_lambda_min
lmax = SPHEREx_lambda_max
resolving_power = SPHEREx_R

Texp = int(180 * 364/14 * 5 * 0.5)
Texp

11700

In [3]:
from svnds.info_VIKING import NAMES, FILTERS, COL_WAVE, COL_RESP

In [4]:
tbl_7ds40 = Table.read("/data8/EL_COSMOS/synphots/SDS/original_eff/synphot_7ds_yrs_all_eff_tot.csv")

In [5]:
tbl_VIKING = Table.read("/data8/EL_COSMOS/synphots/VIKING/synphot_VIKING_tot.csv")
# tbl_PS = Table.read("/data8/EL_COSMOS/synphots/PANSTARRS/synphot_PANSTARRS_tot.csv")
# tbl_spx = Table.read("/data8/EL_COSMOS/synphots/SPHEREx_m6250/scale_22.5/synphot_SPHEREx_scale_22.5_tot.csv")

In [6]:
# tbl_spx = Table.read("/data8/EL_COSMOS/synphots/SPHEREx/scale_22.5/synphot_SPHEREx_scale_22.5_tot.csv")

In [7]:
nsamp = len(tbl_7ds40)
nsamp

518404

In [8]:
# header
header = '#'
header += '\t' + 'id'

### 7DS 40 filters
for ii, wl_cen in enumerate(lambda_7ds):

    wave_cen = int(wl_cen)
    
    name = f'F_F{wave_cen:d}W250'
    header += '\t' + name
    
    name = f'E_F{int(wl_cen):d}W250'
    header += '\t' + name

### VIKING
for ii, wl_cen in enumerate(NAMES):
    
    wave_cen = wl_cen
    
    name = 'F_' + wave_cen 
    header += '\t' + name
    
    name = 'E_' + wave_cen
    header += '\t' + name

### PS
# for ii, wl_cen in enumerate(NAMES):
    
#     wave_cen = wl_cen
    
#     name = 'F_' + wave_cen 
#     header += '\t' + name
    
#     name = 'E_' + wave_cen
#     header += '\t' + name

### SPHEREx
# for iband, (l1, l2, R) in enumerate(zip(lmin, lmax, resolving_power)):
#     i_steps  = np.arange(16, dtype=float)
#     lambda_i = l1 * (((2*R+1)/(2*R-1))**i_steps)
    
#     for ii, wl_cen in enumerate(lambda_i):
#         name = f'F_band{iband+1:d}_{ii+1}'
#         header += '\t' + name

#         name = f'E_band{iband+1:d}_{ii+1}'
#         header += '\t' + name

header += '\n'

In [9]:
values = ''

for i in range(nsamp):
    
    #ID = spec
    spec = tbl_7ds40['spec'][i]
    
    val = str(spec) + '\t'
#     val = ''
    
    for ii, wl_cen in enumerate(lambda_7ds):
        wave_cen = f'{int(wl_cen):d}'
        col = 'flux_' + wave_cen
        
        #flux
        fl = tbl_7ds40[i][col]
        #err
        err = tbl_7ds40[i][col + f'_err_{Texp:d}']
        add_err = np.random.normal( fl, err )

        fl = add_err
        
        val += f'{fl:.5E}' + '\t'
        val += f'{err:.5E}' + '\t'

    # ## PS
    # for ii, wl_cen in enumerate(NAMES):

    #     wave_cen = wl_cen
        
    #     col = 'flux_' + wave_cen 
    #     fl = tbl_PS[i][col]
        
    #     col = 'flux_' + wave_cen + '_err'
    #     err = tbl_PS[i][col]
    #     add_err = np.random.normal( fl, err )
        
    #     fl = add_err
        
    #     val += f'{fl:.5E}' + '\t'
    #     val += f'{err:.5E}' + '\t'
    
    ### VIKING
    for ii, wl_cen in enumerate(NAMES):

        wave_cen = wl_cen
        
        col = 'flux_' + wave_cen 
        fl = tbl_VIKING[i][col]
        
        col = 'flux_' + wave_cen + '_err'
        err = tbl_VIKING[i][col]
        add_err = np.random.normal( fl, err )
        
        fl = add_err
        
        val += f'{fl:.5E}' + '\t'
        val += f'{err:.5E}' + '\t'
    
   ### 7DS SPHEREx
    # for iband, (l1, l2, R) in enumerate(zip(lmin, lmax, resolving_power)):
    #     i_steps  = np.arange(16, dtype=float)
    #     lambda_i = l1 * (((2*R+1)/(2*R-1))**i_steps)

    #     for ii, wl_cen in enumerate(lambda_i):
            
    #         fl = tbl_spx[i][f'flux_band{iband+1:d}_{ii+1}']
    #         err = tbl_spx[i][f'flux_band{iband+1:d}_{ii+1}_err']
            
    #         add_err = np.random.normal( fl, err )
    #         fl = add_err
            
    #         val += f'{fl:.5E}' + '\t'
    #         val += f'{err:.5E}' + '\t'
    

#     zphot = cosmos['ZPHOT'][i]
#     val += f'{zphot:.5E}' + '\t'
    
    val += '\n'
    values += val

In [15]:
# with open("/data8/EL_COSMOS/EAZY/realization/SPHEREx/" + f'input_elcosmos_sds40_WFS_5yr_eff_SPHEREx_scale_22.5_adderr_0.cat', 'w') as f:
    
#     f.write(header + values)

In [12]:
with open("/data8/EL_COSMOS/EAZY/realization/VIKING/" + f'input_elcosmos_sds40_WFS_5yr_eff_VIKING_adderr_0.cat', 'w') as f:
    
    f.write(header + values)

In [10]:
1

1